# SQL Khan Academy Tutorial

### Intro

Set up the database:

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('sample_data/khan1.db')
cur = conn.cursor()

Create a function for easy printing of a query into a dataframe (note, the pd.read_sql() function executes the query for me):

In [ ]:
def df_maker(query):
    df = pd.read_sql_query(query, conn)
    return df

My data:

In [ ]:
"""
Grocery list:
Bananas (4)
Peanut Butter (1)
Dark Chocolate Bars (2)
"""

Add the schema:

In [ ]:
new_schema = """
CREATE TABLE groceries
(id INTEGER PRIMARY KEY, name TEXT, quantity INTEGER, aisle INTEGER)
"""

cur.execute(new_schema)
conn.commit()

Add some data:

In [ ]:
add_to_db = [
    "INSERT INTO groceries VALUES (1, 'Bananas', 4, 7)",
    "INSERT INTO groceries VALUES (2, 'Peanut Butter', 1, 2)",
    "INSERT INTO groceries VALUES (3, 'Dark chocolate bars', 2, 2)",
    "INSERT INTO groceries VALUES (4, 'Ice creame', 1, 12)",
    "INSERT INTO groceries VALUES (5, 'Cherries', 6, 2)",
    "INSERT INTO groceries VALUES (6, 'Chocolate syrup', 1, 4)"
]

for query in add_to_db:
    cur.execute(query)
    conn.commit()

Use query to show the data:

In [ ]:
query = """
SELECT * FROM groceries
"""

table = df_maker(query)
table

Show all rows with aisle greater than 5. Order by aisle number:

In [ ]:
query = """
SELECT *
FROM groceries
WHERE aisle > 5
ORDER BY aisle DESC
"""

df_maker(query)

Now sum up all the items in the quantity field:

In [ ]:
query = """
SELECT
    SUM(quantity)
FROM groceries
"""

df_maker(query)

Find the item with the greatest quantity:

In [ ]:
query = """
SELECT
    MAX(quantity)
FROM groceries
"""

df_maker(query)

Find number of items per aisle:

In [ ]:
query = """
SELECT
    aisle,
    SUM(quantity)
FROM groceries
GROUP BY aisle
"""

df_maker(query)

Note, selecting a different item than what is being grouped by may give a nosensical result. First, remember the main table:

In [ ]:
table

In [ ]:
query = """
SELECT
    name,
    SUM(quantity)
FROM groceries
GROUP BY aisle
"""

df_maker(query)

But the SQL engine only selects the top-most name to represent what I'm grouping by - though the SUM(quantity) is actually the total items per aisle, regardless of what they are.

In [ ]:
conn.close()

### More Complex Queries

Set up a new database:

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('sample_data/khan2.db')
cur = conn.cursor()

Function to load and view table into a dataframe:

In [ ]:
import pandas as pd

def df_maker(query):
    df = pd.read_sql(query, conn)
    return df

Add schema:

In [ ]:
new_schema = """
CREATE TABLE exercise_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    minutes INTEGER, 
    calories INTEGER,
    heart_rate INTEGER)
"""

cur.execute(new_schema)
conn.commit()

Add some data:

In [ ]:
add_to_db = [
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('biking', 30, 100, 110)",
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('biking', 10, 30, 105)",
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('dancing', 15, 200, 120)",
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('tree climbing', 30, 70, 90)",
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('tree climbing', 25, 72, 80)",
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('rowing', 30, 70, 90)",
    "INSERT INTO exercise_logs(type, minutes, calories, heart_rate) VALUES ('hiking', 60, 80, 85)"
]

for query in add_to_db:
    cur.execute(query)
    conn.commit()

View the table:

In [ ]:
exercise_logs_query = """
SELECT * FROM exercise_logs
"""

exercise_logs = df_maker(exercise_logs_query)
exercise_logs

Filter where calories greater than 50. Order by calories:

In [ ]:
query = """
SELECT *
FROM exercise_logs
WHERE calories > 50
ORDER BY calories
"""

df_maker(query)

Chain together conditions with the AND operator:

In [ ]:
query = """
SELECT *
FROM exercise_logs
WHERE calories > 50 AND minutes < 30
"""

df_maker(query)

Now with the OR operator:

In [ ]:
query = """
SELECT *
FROM exercise_logs
WHERE calories > 50 OR heart_rate > 100
"""

df_maker(query)

Find all matching a list using the IN operator:

In [ ]:
query = """
SELECT *
FROM
    exercise_logs
WHERE type IN
    (
    'biking',
    'hiking',
    'tree climbing',
    'rowing'
    )
"""

df_maker(query)

Can also use NOT IN to do the opposite:

In [ ]:
query = """
SELECT *
FROM
    exercise_logs
WHERE type NOT IN
    (
    'biking',
    'hiking',
    'tree climbing',
    'rowing'
    )
"""

df_maker(query)

Now create a new table with doctor-recommended activities:

In [ ]:
new_schema = """
CREATE TABLE drs_favorites
    (id INTEGER PRIMARY KEY,
    type TEXT,
    reason TEXT);
"""

cur.execute(new_schema)
conn.commit()

Add some data:


In [ ]:
add_to_db = [
    "INSERT INTO drs_favorites(type, reason) VALUES ('biking', 'Improves endurance and flexibility.')",
    "INSERT INTO drs_favorites(type, reason) VALUES ('hiking', 'Increases cardiovascular health.')"
]

for query in add_to_db:
    cur.execute(query)
    conn.commit()

In [ ]:
drs_favorites_query = """
SELECT * FROM drs_favorites
"""

drs_favorites = df_maker(drs_favorites_query)
drs_favorites

Now show all exercise logs corresponding to doctor-recommended activities using a sub-query. Remember exercise_logs first:

In [ ]:
exercise_logs

In [ ]:
query = """
SELECT *
FROM
    exercise_logs
WHERE type IN (
    SELECT type
    FROM drs_favorites
)
"""

df_maker(query)

Use a subquery to search for a category that contains the word 'cardiovascular':

In [ ]:
query = """
SELECT * 
FROM exercise_logs WHERE type IN (
    SELECT type 
    FROM drs_favorites
    WHERE reason LIKE "%cardiovascular%")
"""

df_maker(query)

Now use HAVING to see how many calories burned per type of exercise:

In [ ]:
query = """
SELECT
    type,
    SUM(calories) AS total_calories
FROM exercise_logs
GROUP BY type
"""

df_maker(query)

Now only show activities where more than 150 are burned:

In [ ]:
query = """
SELECT
    type,
    SUM(calories) AS total_calories
FROM exercise_logs
GROUP BY type
HAVING total_calories > 140
"""

df_maker(query)

Average calories for each type of exercise greater than a certain amount:

In [ ]:
query = """
SELECT
    type,
    AVG(calories) AS avg_calories
FROM exercise_logs
GROUP BY type
HAVING avg_calories > 70
"""

df_maker(query)

Find all exercises where heart_rate exceeds my maximum heart rate for my age (there should be none...):

In [ ]:
query = """
SELECT *
FROM exercise_logs
WHERE heart_rate > 220 - 36
"""

df_maker(query)

Now find where heart_rate is between 50 and 90% of max:

In [ ]:
query = """
SELECT *
FROM exercise_logs
WHERE
    heart_rate >= ROUND(0.50 * (220-36))
    AND
    heart_rate <= ROUND(0.90 * (220-36))
"""

df_maker(query)

Also can just count how many activities fall into this category:

In [ ]:
query = """
SELECT COUNT(*)
FROM exercise_logs
WHERE
    heart_rate >= ROUND(0.50 * (220-36))
    AND
    heart_rate <= ROUND(0.90 * (220-36))
"""

df_maker(query)

Categorize each activity according to which zone the heart is in. First, narrow down exercise_logs table:

In [ ]:
query = """
SELECT
    type,
    heart_rate
FROM exercise_logs
"""

df_maker(query)

Create a new column with a category for each row:

In [ ]:
query = """
SELECT
    type,
    heart_rate,
CASE
    WHEN heart_rate > 220-30 THEN "above max"
    WHEN heart_rate > ROUND(0.90 * (220-30)) THEN "above target"
    WHEN heart_rate > ROUND(0.50 * (220-30)) THEN "within target"
    ELSE "below target"
END as "hr_zone"
FROM exercise_logs
"""

df_maker(query)

Now do the same thing but group by a certain HR zone:

In [ ]:
query = """
SELECT
    type,
    heart_rate,
CASE
    WHEN heart_rate > 220-30 THEN "above max"
    WHEN heart_rate > ROUND(0.90 * (220-30)) THEN "above target"
    WHEN heart_rate > ROUND(0.50 * (220-30)) THEN "within target"
    ELSE "below target"
END as "hr_zone"
FROM exercise_logs
GROUP BY hr_zone
"""

df_maker(query)

### Joins

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("sample_data/khan3.db")
c = conn.cursor()

def df_maker(query):
    df = pd.read_sql(query, conn)
    return df

In [2]:
new_schema = [
"""
CREATE TABLE students (id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    birthdate TEXT,
    buddy_id INTEGER);
""",
"""
CREATE TABLE student_grades (id INTEGER PRIMARY KEY,
    student_id INTEGER,
    test TEXT,
    grade INTEGER)
""",
"""
CREATE TABLE student_projects (id INTEGER PRIMARY KEY,
    student_id INTEGER,
    title TEXT)
""",
"""
CREATE TABLE project_pairs (id INTEGER PRIMARY KEY,
    project1_id INTEGER,
    project2_id INTEGER)
"""
]
for schema in new_schema:
    c.execute(schema)

new_rows = [
    "INSERT INTO students VALUES (1, 'Peter','Rabbit', 'peter@rabbit.com', '555-6666','2002-06-24', 2)",
    "INSERT INTO students VALUES (2, 'Alice', 'Wonderland', 'alice@wonderland.com', '555-4444','2002-07-04', 1)",
    "INSERT INTO students VALUES (3, 'Aladdin', 'Lampland', 'aladdin@lampland.com', '555-3333', '2001-05-10', 4)",
    "INSERT INTO students VALUES (4, 'Simba', 'Kingston', 'simba@kingston.com', '555-1111', '2001-12-24', 3)",
    "INSERT INTO student_grades (student_id, test, grade) VALUES (1, 'Nutrition', 95)",
    "INSERT INTO student_grades (student_id, test, grade) VALUES (2, 'Nutrition', 92)",
    "INSERT INTO student_grades (student_id, test, grade) VALUES (1, 'Chemistry', 85)",
    "INSERT INTO student_grades (student_id, test, grade) VALUES (2, 'Chemistry', 95)",
    "INSERT INTO student_projects (student_id, title) VALUES (1, 'Carrotapault')",
    "INSERT INTO student_projects (student_id, title) VALUES (2, 'Mad Hattery')",
    "INSERT INTO student_projects (student_id, title) VALUES (3, 'Carpet Physics')",
    "INSERT INTO project_pairs (project1_id, project2_id) VALUES(1, 2)",
    "INSERT INTO project_pairs (project1_id, project2_id) VALUES(3, 4)"
]
for row in new_rows:
    c.execute(row)
    

# "INSERT INTO student_projects (student_id, title) VALUES (4, 'Hyena Habitats')",

Check out my tables:

In [3]:
query = """
SELECT * FROM students
"""

df_maker(query)

,id,first_name,last_name,email,phone,birthdate,buddy_id
0,1,Peter,Rabbit,peter@rabbit.com,555-6666,2002-06-24,2
1,2,Alice,Wonderland,alice@wonderland.com,555-4444,2002-07-04,1
2,3,Aladdin,Lampland,aladdin@lampland.com,555-3333,2001-05-10,4
3,4,Simba,Kingston,simba@kingston.com,555-1111,2001-12-24,3


In [4]:
query = """
SELECT * FROM student_grades
"""

df_maker(query)

,id,student_id,test,grade
0,1,1,Nutrition,95
1,2,2,Nutrition,92
2,3,1,Chemistry,85
3,4,2,Chemistry,95


In [5]:
query = """
SELECT * FROM student_projects
"""

df_maker(query)

,id,student_id,title
0,1,1,Carrotapault
1,2,2,Mad Hattery
2,3,3,Carpet Physics


In [6]:
query = """
SELECT * FROM project_pairs
"""

df_maker(query)

,id,project1_id,project2_id
0,1,1,2
1,2,3,4


Now make a cross join (note, the following is an 'implicit' inner join):

In [7]:
query = """
SELECT * FROM student_grades, students
"""

df_maker(query)

,id,student_id,test,grade,id,first_name,last_name,email,phone,birthdate,buddy_id
0,1,1,Nutrition,95,1,Peter,Rabbit,peter@rabbit.com,555-6666,2002-06-24,2
1,1,1,Nutrition,95,2,Alice,Wonderland,alice@wonderland.com,555-4444,2002-07-04,1
2,1,1,Nutrition,95,3,Aladdin,Lampland,aladdin@lampland.com,555-3333,2001-05-10,4
3,1,1,Nutrition,95,4,Simba,Kingston,simba@kingston.com,555-1111,2001-12-24,3
4,2,2,Nutrition,92,1,Peter,Rabbit,peter@rabbit.com,555-6666,2002-06-24,2
5,2,2,Nutrition,92,2,Alice,Wonderland,alice@wonderland.com,555-4444,2002-07-04,1
6,2,2,Nutrition,92,3,Aladdin,Lampland,aladdin@lampland.com,555-3333,2001-05-10,4
7,2,2,Nutrition,92,4,Simba,Kingston,simba@kingston.com,555-1111,2001-12-24,3
8,3,1,Chemistry,85,1,Peter,Rabbit,peter@rabbit.com,555-6666,2002-06-24,2
9,3,1,Chemistry,85,2,Alice,Wonderland,alice@wonderland.com,555-4444,2002-07-04,1


But this is not useful. what we want is for the sudent_id from student_grades to match to id from students. So do inner join by putting student_grades first and imposing the restriction using the WHERE statement (notice, left table rows are duplicated where needed to match a row on the right):

In [8]:
query = """
SELECT * FROM student_grades, students
WHERE
    student_grades.student_id=students.id
"""

df_maker(query)

,id,student_id,test,grade,id,first_name,last_name,email,phone,birthdate,buddy_id
0,1,1,Nutrition,95,1,Peter,Rabbit,peter@rabbit.com,555-6666,2002-06-24,2
1,2,2,Nutrition,92,2,Alice,Wonderland,alice@wonderland.com,555-4444,2002-07-04,1
2,3,1,Chemistry,85,1,Peter,Rabbit,peter@rabbit.com,555-6666,2002-06-24,2
3,4,2,Chemistry,95,2,Alice,Wonderland,alice@wonderland.com,555-4444,2002-07-04,1


Now do an explicit inner join:

In [9]:
query = """
SELECT
    students.first_name,
    students.last_name,
    students.email,
    student_grades.test,
    student_grades.grade
FROM students
    JOIN student_grades
        ON students.id=student_grades.student_id
"""

df_maker(query)

,first_name,last_name,email,test,grade
0,Peter,Rabbit,peter@rabbit.com,Nutrition,95
1,Alice,Wonderland,alice@wonderland.com,Nutrition,92
2,Peter,Rabbit,peter@rabbit.com,Chemistry,85
3,Alice,Wonderland,alice@wonderland.com,Chemistry,95


Again, note that where there is a match, data from the lead table (students) will be duplicated to align to the trailing table. Now only show grades above 90:

In [10]:
query = """
SELECT
    students.first_name,
    students.last_name,
    students.email,
    student_grades.test,
    student_grades.grade
FROM students
    JOIN student_grades
        ON students.id=student_grades.student_id
WHERE student_grades.grade > 90
"""

df_maker(query)

,first_name,last_name,email,test,grade
0,Peter,Rabbit,peter@rabbit.com,Nutrition,95
1,Alice,Wonderland,alice@wonderland.com,Nutrition,92
2,Alice,Wonderland,alice@wonderland.com,Chemistry,95


Now inner join to find students and their projects:

In [11]:
query = """
SELECT
    students.first_name,
    students.last_name,
    student_projects.title
FROM students
    JOIN student_projects
        ON students.id = student_projects.student_id
"""

df_maker(query)

,first_name,last_name,title
0,Peter,Rabbit,Carrotapault
1,Alice,Wonderland,Mad Hattery
2,Aladdin,Lampland,Carpet Physics


But now we're only showing people with a project. What if we want everyone, and the people with a project? A left outer join can do this:

In [12]:
query = """
SELECT
    students.first_name,
    students.last_name,
    student_projects.title
FROM students
    LEFT OUTER JOIN student_projects
        ON students.id = student_projects.student_id
"""

df_maker(query)

,first_name,last_name,title
0,Peter,Rabbit,Carrotapault
1,Alice,Wonderland,Mad Hattery
2,Aladdin,Lampland,Carpet Physics
3,Simba,Kingston,None


Note, switching table order when making a join is the same as doing a right outer join. Let's show the names of the students next to the email of the buddy. So join a table to itself. Will need to use an alias. Notice that if no table exists, SQL will assume it's an alias for the current table with the matching field:

In [14]:
query = """
SELECT
    students.first_name,
    students.last_name,
    non_existant_buddy_alias.email as buddy_email
FROM students
    JOIN students non_existant_buddy_alias
        ON students.buddy_id = non_existant_buddy_alias.id
"""

df_maker(query)

,first_name,last_name,buddy_email
0,Peter,Rabbit,alice@wonderland.com
1,Alice,Wonderland,peter@rabbit.com
2,Aladdin,Lampland,simba@kingston.com
3,Simba,Kingston,aladdin@lampland.com


The project_pairs table represents project-pairs which will indicate to students who they should team up with the review the project:

In [15]:
query = """
SELECT * FROM project_pairs
"""

df_maker(query)

,id,project1_id,project2_id
0,1,1,2
1,2,3,4


In [16]:
query = """
SELECT * FROM student_projects
"""

df_maker(query)

,id,student_id,title
0,1,1,Carrotapault
1,2,2,Mad Hattery
2,3,3,Carpet Physics


The goal here is to map the numbers in project_pairs to their corresponding project name:

In [17]:
query = """
SELECT
    a.title,
    b.title
FROM project_pairs
    JOIN student_projects a
    ON project_pairs.project1_id = a.id
    JOIN student_projects b
    ON project_pairs.project2_id = b.id
"""

df_maker(query)

,title,title
0,Carrotapault,Mad Hattery
